In [2]:
# 解凍
import shutil
import zipfile

shutil.unpack_archive('/workspace/ssd4t/yoshikawa/241030_cell_knowledge_ext/for_azuma.zip', '/workspace/ssd4t/yoshikawa/241030_cell_knowledge_ext/copy')

In [6]:
# predication を 50000 行に区切る

input_file = "/workspace/ssd4t/yoshikawa/0_DATA/SemMed/00_data_from_semmed/semmedVER43_2024_R_PREDICATION.csv"
output_file = "/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/csv/%d_semmed_pred.csv"

line_max = 50000

line_index = 1
file_seqno = 1

with open(input_file, "r", encoding="utf-8", errors="ignore") as in_file:
    out_file = open(output_file % file_seqno, "w", encoding="utf-8")

    for line in in_file:
        out_file.write(line)
        line_index += 1

        if line_index > line_max:
            out_file.close()
            file_seqno += 1
            line_index = 1
            out_file = open(output_file % file_seqno, "w", encoding="utf-8")

    out_file.close()

In [32]:
import pandas as pd

df = pd.read_csv("/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_sent/csv/25_semmed_sent.csv", on_bad_lines="skip").head()
print(df)

   1381174  3685099  ti  1    20  \
0  1381175  4859544  ti  3    89   
1  1381176  3865199  ti  1    20   
2  1381177  3954747  ab  8  1268   
3  1381178  3474940  ab  2   412   
4  1381179  5130218  ab  2   234   

  Random selection of points distributed on curved surfaces.    79  \
0  Production of fluorogibberellic acid and fluor...           174   
1  Stereochemical course of the reaction catalyze...           156   
2  The carcinoma itself showed rapid rates of gly...          1379   
3  These cell suspensions were implanted unilater...           578   
4  Out of 130 patients presenting 78 were selecte...           318   

   Unnamed: 7  Unnamed: 8  
0         NaN         NaN  
1         NaN         NaN  
2         NaN         NaN  
3         NaN         NaN  
4         NaN         NaN  


In [1]:
import pandas as pd

df = pd.read_csv("/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/csv/1_semmed_pred.csv").head()
print(df)

   10592604  16  16530475  PROCESS_OF  C0003725               Arboviruses  \
0  10592697  17  16530475         ISA  C0039258              Tahyna virus   
1  10592728  17  16530475         ISA  C0318627               Eyach virus   
2  10592759  17  16530475         ISA  C0446169  California Group Viruses   
3  10592832  18  16530475  PROCESS_OF  C0012634                   Disease   
4  10592873  18  16530475      CAUSES  C0042776                     Virus   

   virs  1  C0999630            Lepus capensis  mamm  1.1  \N \N.1 \N.2  
0  virs  1  C0446169  California Group Viruses  virs    1  \N   \N   \N  
1  virs  1  C0206590                Coltivirus  virs    1  \N   \N   \N  
2  virs  1  C0003725               Arboviruses  virs    1  \N   \N   \N  
3  dsyn  0  C0020114                     Human  humn    0  \N   \N   \N  
4  virs  1  C0012634                   Disease  dsyn    0  \N   \N   \N  


In [ ]:
# tsv を db に変換
import csv, sqlite3

def tsv_to_db(file_number):
    conn = sqlite3.connect(f"/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/db/{file_number}_semmed_pred.db")
    curs = conn.cursor()
    curs.execute(f"CREATE TABLE IF NOT EXISTS raw (pred_id, sent_id, pmid, predicate, sub_name, sub_ty, obj_name, obj_ty);")
    
    reader = csv.reader(open(f"/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/csv/{file_number}_semmed_pred.csv", 'r'))
    next(reader)

    for row in reader:
        to_db = [row[0], row[1], row[2], row[3], row[5], row[6], row[9], row[10]]
        curs.execute(f"INSERT INTO raw (pred_id, sent_id, pmid, predicate, sub_name, sub_ty, obj_name, obj_ty) VALUES (?, ?, ?, ?, ?, ?, ?, ?);", to_db)

    conn.commit()
    conn.close()

for i in range(1, 2611):
    tsv_to_db(i)

In [12]:
# 複数の csv を 1 つの db に変換

import csv, sqlite3

def tsv_to_db(file_number):
    conn = sqlite3.connect("/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_sent/db/semmed_sent.db")
    curs = conn.cursor()
    curs.execute(f"CREATE TABLE IF NOT EXISTS raw (sent_id, pmid, type, sentence);")
    
    reader = csv.reader(open(f"/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_sent/csv/{file_number}_semmed_sent.csv", 'r'))
    next(reader)

    for row in reader:
        to_db = [row[0], row[1], row[2], row[5]]
        curs.execute(f"INSERT INTO raw (sent_id, pmid, type, sentence) VALUES (?, ?, ?, ?);", to_db)

    conn.commit()
    conn.close()

for i in range(2611, 5265):
    tsv_to_db(i)

In [ ]:
import sqlite3

# 高速化のため、index を作成
def add_index_to_sentence_db():
    conn = sqlite3.connect('/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_sent/db/semmed_sent.db')
    cursor = conn.cursor()
    
    cursor.execute('CREATE INDEX IF NOT EXISTS idx_sentence_sentid_pmid ON raw (sent_id, pmid)')

    conn.commit()
    conn.close()

add_index_to_sentence_db()

In [ ]:
def add_index_to_raw(file_number):
    conn = sqlite3.connect(f'/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/db/{file_number}_semmed_pred.db')
    cursor = conn.cursor()

    cursor.execute('CREATE INDEX IF NOT EXISTS idx_raw_sentid_pmid ON raw (sent_id, pmid)')

    conn.commit()
    conn.close()

for i in range(1, 2611):
    add_index_to_raw(i)

In [5]:
# predication と sentence の結合

import sqlite3

def combine_pred_sent(file_number):
    # メインのデータベースに接続
    conn = sqlite3.connect(f'/workspace/ssd4t/yoshikawa/0_DATA/SemMed/02_semmed_pred/db/{file_number}_semmed_pred.db')
    cursor = conn.cursor()

    # 別のデータベースをATTACHで追加
    cursor.execute("ATTACH DATABASE '/workspace/ssd4t/yoshikawa/0_DATA/SemMed/01_semmed_sent/db/semmed_sent.db' AS sentence_db")

    # 新しいテーブルを作成し、結合結果を挿入
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS pred_sent AS
        SELECT 
            p.pred_id,
            p.sent_id,
            p.pmid,
            p.predicate,
            p.sub_name,
            p.obj_name,
            s.sentence
        FROM 
            raw p
        LEFT JOIN 
            sentence_db.raw s
        ON 
            p.sent_id = s.sent_id AND p.pmid = s.pmid
    ''')

    # コミットしてデータベース接続を閉じる
    conn.commit()
    conn.close()

for i in range(1, 2611):
    combine_pred_sent(i)